In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing

In [3]:
data = pd.read_csv('data_clean.csv')

In [4]:
data.head()

,Unnamed: 0,X,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,EnclosedPorch,X3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,0,1,60,RL,65.0,8450,Pave,No Alley,Reg,...,0,0,0,No Fence,0,2,2008,WD,Normal,208500.0
1,2,1,2,20,RL,80.0,9600,Pave,No Alley,Reg,...,0,0,0,No Fence,0,5,2007,WD,Normal,181500.0
2,3,2,3,60,RL,68.0,11250,Pave,No Alley,IR1,...,0,0,0,No Fence,0,9,2008,WD,Normal,223500.0
3,4,3,4,70,RL,60.0,9550,Pave,No Alley,IR1,...,272,0,0,No Fence,0,2,2006,WD,Abnorml,140000.0
4,5,4,5,60,RL,84.0,14260,Pave,No Alley,IR1,...,0,0,0,No Fence,0,12,2008,WD,Normal,250000.0


In [5]:
train = data[0:1460]
test = data[1461:]
test = test.drop(["SalePrice"], axis = 1)

In [6]:
train_neigh1 = train[['Neighborhood', 'SalePrice']]
train_neigh1.head()

,Neighborhood,SalePrice
0,CollgCr,208500.0
1,Veenker,181500.0
2,CollgCr,223500.0
3,Crawfor,140000.0
4,NoRidge,250000.0


Assigning our own labels to each neighborhood

In [7]:
tn = train.Neighborhood.unique()
tn = pd.DataFrame(tn)
tn['mylablels']= list(range(1,26))
tn.columns = ['Neighborhood', 'mylabels']
train_neigh1 = train_neigh1.merge(tn, on = "Neighborhood")
train_neigh1 = train_neigh1[["mylabels", "SalePrice"]]

Aggregating by mean price per neighborhood

In [8]:
train_neigh_means = train_neigh1.groupby("mylabels").agg("mean").reset_index().sort_values("SalePrice")

In [9]:
#LABEL ENCODING
#train_neigh = pd.get_dummies(train_neigh1['Neighborhood'])
#train_neigh['SalePrice'] = train_neigh1.SalePrice

In [10]:
train_neigh_means = train_neigh_means.to_numpy()

Fitting k means

In [11]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(train_neigh_means)
klabels = kmeans.labels_
klabels

array([3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4,
       1, 1, 1], dtype=int32)

In [12]:
klabels = pd.DataFrame(klabels)

In [13]:
ktrain_neigh = pd.DataFrame(train_neigh_means)
ktrain_neigh['klabels'] = klabels

In [14]:
#DE-DUMMIFYING 
#ktrain_neigh['nbrhd'] = ""
#for col in ktrain_neigh.columns[:25]:
#    indices =  ktrain_neigh.loc[train_neigh[col]==1].index
#    for j in indices:
#        ktrain_neigh['nbrhd'][j] = col

In [15]:
#ktrain_neigh = ktrain_neigh[['nbrhd', 'klabels', 'SalePrice']]

In [16]:
ktrain_neigh.columns = ['mylabels', 'SalePrice', 'klabels']

In [22]:
ktrain_neigh.groupby("klabels").agg({"SalePrice":"mean", "mylabels":"nunique"}).reset_index().sort_values("SalePrice").rename(columns={"mylabels": "nbrhdCount", "SalePrice" : "meanSalePrice"})

,klabels,meanSalePrice,nbrhdCount
3,3,101064.668124,3
2,2,138108.354960,9
0,0,197783.883026,7
4,4,235466.670617,3
1,1,320688.313483,3


In [18]:
train_neigh_dict = ktrain_neigh[['klabels', 'mylabels']]
train_neigh_dict = ktrain_neigh.drop_duplicates()
train_neigh_dict = train_neigh_dict.set_index('klabels').sort_values("SalePrice").reset_index()
train_neigh_dict = train_neigh_dict.merge(tn, on= 'mylabels')[['klabels','Neighborhood','SalePrice']]
train_neigh_dict

,klabels,Neighborhood,SalePrice
0,3,MeadowV,98576.470588
1,3,IDOTRR,100123.783784
2,3,BrDale,104493.750000
3,2,BrkSide,124834.051724
4,2,Edwards,128219.700000
5,2,OldTown,128225.300885
6,2,Sawyer,136793.135135
7,2,Blueste,137500.000000
8,2,SWISU,142591.360000
9,2,NPkVill,142694.444444
